## Using Python to Integrate MongoDB Data into an ETL Process
This notebook demonstrates the setup of an ETL (Extract, Transform, Load) pipeline.

In this lab you will build upon the **Northwind_DW2** dimensional database from Lab 3; however, you will be integrating new data sourced from an instance of MongoDB. The new data will be concerned with new business processes; inventory and purchasing. You will continue to interact with both the source systems (MongoDB and MySQL), and the destination system (the Northwind_DW2 data warehouse) from a remote client running Python (Jupyter Notebooks). 

I fetch data into Pandas DataFrames, perform all the necessary transformations in-memory on the client, and then push the newly transformed DataFrame to the RDBMS data warehouse using a Pandas function that will create the table and fill it with data with a single operation.

### Prerequisites:
This notebook uses the PyMongo database connectivity library to connect to MySQL databases; therefore, you must have first installed that libary into your python environment by executing the following command in a Terminal window.

- `python -m pip install pymongo[srv]`

#### Import the Necessary Libraries

In [28]:
import os
import logging
from typing import Dict
import pandas as pd
import numpy as np
import pymysql


import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

In [24]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.34
Running PyMongo Version: 4.8.0


#### Declare & Assign Connection Variables for the MongoDB Server, the MySQL Server & Databases with which You'll be Working 

In [25]:
from pymongo import MongoClient
import json

# Example setup of logging for the notebook
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Function to get MongoDB client
def get_mongo_client(host: str, port: int, username: str = None, password: str = None) -> MongoClient:
    """Initialize MongoDB client."""
    if username and password:
        client = MongoClient(host, port, username=username, password=password)
    else:
        client = MongoClient(host, port)
    logger.info("MongoDB client initialized.")
    return client


# SQL connection
def get_sql_connection(host: str, user: str, password: str, db: str):
    """Initialize SQL connection."""
    conn = pymysql.connect(host=host, user=user, password=password, db=db)
    return conn

In [27]:
# Set the path of the current working directory and append 'data' directory
data_dir = os.path.join(os.getcwd(), 'data')
logger.info(f"Data directory set to: {data_dir}")


FileNotFoundError: [Errno 2] No such file or directory

In [21]:
# Define JSON files for MongoDB collections
json_files = {
    "sales_orders": 'StoreSales.json',
}


In [19]:
def set_mongo_collections(client: MongoClient, db_name: str, data_dir: str, json_files: dict):
    """Load JSON data into MongoDB collections."""
    db = client[db_name]
    for collection_name, file_name in json_files.items():
        file_path = os.path.abspath(os.path.join(data_dir, file_name))
        
        # Load JSON data and insert into MongoDB
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
                if isinstance(data, list): 
                    db[collection_name].insert_many(data)
                    logger.info(f"Inserted {len(data)} documents into '{collection_name}' collection.")
                else:
                    db[collection_name].insert_one(data)
                    logger.info(f"Inserted a single document into '{collection_name}' collection.")
        except Exception as e:
            logger.error(f"Error loading data for collection '{collection_name}': {str(e)}")


In [20]:
# MongoDB connection arguments (example)
mongodb_args = {
    "host": "localhost",
    "port": 27017,
    "username": "your_username",
    "password": "your_password",
    "db_name": "northwind_db"
}

# Initialize the MongoDB client
client = get_mongo_client(
    host=mongodb_args["host"],
    port=mongodb_args["port"],
    username=mongodb_args.get("username"),
    password=mongodb_args.get("password")
)

INFO:__main__:MongoDB client initialized.


#### Populate MongoDB with Source Data
You only need to run this cell once; however, the operation is *idempotent*.  In other words, it can be run multiple times without changing the end result.

In [ ]:
# Load data into MongoDB collections
set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)

#### Data Extractor
This class provides mock methods to:

Extract data from a MongoDB collection.

In [9]:

# Setup logging for Jupyter
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Mock DatabaseConnection class (replace with actual class if available)
class DatabaseConnection:
    def get_mongo_connection(self):
        # Return a mock MongoDB connection or client
        logger.info("Mock MongoDB connection created.")
        return {
            'my_collection': [
                {'_id': 1, 'name': 'Alice', 'age': 30},
                {'_id': 2, 'name': 'Bob', 'age': 25}
            ]
        }
    
    def get_api_session(self):
        import requests
        session = requests.Session()
        return session
    
    config = {
        'api': {
            'base_url': 'https://api.example.com'
        }
    }

class DataExtractor:
    def __init__(self, db_connection: DatabaseConnection):
        self.db_conn = db_connection
        
    def extract_from_mongodb(self, collection: str, query: Dict = None) -> pd.DataFrame:
        """Extract data from MongoDB collection"""
        try:
            mongo_db = self.db_conn.get_mongo_connection()
            data = mongo_db.get(collection, [])
            return pd.DataFrame(data)
        except Exception as e:
            logger.error(f"Error extracting from MongoDB: {str(e)}")
            raise
            
    def extract_from_api(self, endpoint: str, params: Dict = None) -> pd.DataFrame:
        """Extract data from REST API"""
        try:
            session = self.db_conn.get_api_session()
            api_config = self.db_conn.config['api']
            response = session.get(f"{api_config['base_url']}/{endpoint}", params=params)
            response.raise_for_status()
            return pd.DataFrame(response.json())
        except Exception as e:
            logger.error(f"Error extracting from API: {str(e)}")
            raise

db_conn = DatabaseConnection()
extractor = DataExtractor(db_conn)

# Mock extraction from MongoDB
try:
    df_mongo = extractor.extract_from_mongodb("my_collection")
    print("MongoDB Data:")
    print(df_mongo)
except Exception as e:
    logger.error(f"Error in MongoDB extraction test: {str(e)}")

# Note: For `extract_from_api`, replace with an actual endpoint or mock response as required.


INFO:__main__:Mock MongoDB connection created.


MongoDB Data:
   _id   name  age
0    1  Alice   30
1    2    Bob   25


## Data Loader
This class provides a mock method load_to_warehouse, simulating loading a DataFrame into a data warehouse. It logs the count of rows loaded.

In [12]:

# Assuming DatabaseConnection is defined and accessible in your environment
# Replace with the actual import if needed
# from your_package.utils.database import DatabaseConnection


# Mock DatabaseConnection (replace with actual implementation if available)
class DatabaseConnection:
    def get_sqlalchemy_engine(self):
        # Mock SQLAlchemy engine, replace with actual database URI
        logger.info("Mock SQLAlchemy engine created.")
        return create_engine('sqlite:///:memory:')  # Using an in-memory SQLite database for demonstration

# DataLoader class
class DataLoader:
    def __init__(self, db_connection: DatabaseConnection):
        self.db_conn = db_connection
    
    def load_to_warehouse(self, df: pd.DataFrame, table_name: str, if_exists: str = 'append') -> None:
        """Load DataFrame to data warehouse"""
        try:
            engine = self.db_conn.get_sqlalchemy_engine()
            df.to_sql(
                name=table_name,
                con=engine,
                if_exists=if_exists,
                index=False,
                chunksize=1000
            )
            logger.info(f"Successfully loaded {len(df)} rows to {table_name}")
        except Exception as e:
            logger.error(f"Error loading data to warehouse: {str(e)}")
            raise

# Usage example
db_connection = DatabaseConnection()
data_loader = DataLoader(db_connection)

# Create a sample DataFrame
sample_data = {
    "id": [1, 2, 3],
    "name": ["Alice", "Bob", "Charlie"],
    "age": [25, 30, 35]
}
df_sample = pd.DataFrame(sample_data)

# Load the sample DataFrame into the mock database
data_loader.load_to_warehouse(df_sample, "sample_table")


INFO:__main__:Mock SQLAlchemy engine created.
INFO:__main__:Successfully loaded 3 rows to sample_table


#### Populate MongoDB with Source Data
You only need to run this cell once; however, the operation is *idempotent*.  In other words, it can be run multiple times without changing the end result.

### 1.0. Create and Populate the New Dimension Tables
#### 1.1. Extract Data from the Source MongoDB Collections Into DataFrames

In [1]:

# Set up logging for Jupyter
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# DataTransformer class
class DataTransformer:
    @staticmethod
    def clean_customer_data(df: pd.DataFrame) -> pd.DataFrame:
        """Clean and transform customer data"""
        try:
            # Remove duplicates
            df = df.drop_duplicates()
            
            # Handle missing values
            df['email'] = df['email'].fillna('')
            df['phone'] = df['phone'].fillna('')
            
            # Standardize phone numbers
            df['phone'] = df['phone'].apply(lambda x: ''.join(filter(str.isdigit, str(x))))
            
            # Convert dates to datetime
            df['registration_date'] = pd.to_datetime(df['registration_date'])
            
            logger.info("Customer data cleaned successfully.")
            return df
        except Exception as e:
            logger.error(f"Error cleaning customer data: {str(e)}")
            raise
    
    @staticmethod
    def transform_sales_data(df: pd.DataFrame) -> pd.DataFrame:
        """Transform sales data"""
        try:
            # Calculate derived columns
            df['total_amount'] = df['quantity'] * df['unit_price']
            df['discount_amount'] = df['total_amount'] * df['discount_rate']
            df['final_amount'] = df['total_amount'] - df['discount_amount']
            
            # Convert dates
            df['sale_date'] = pd.to_datetime(df['sale_date'])
            
            # Add time dimensions
            df['sale_year'] = df['sale_date'].dt.year
            df['sale_month'] = df['sale_date'].dt.month
            df['sale_quarter'] = df['sale_date'].dt.quarter
            
            logger.info("Sales data transformed successfully.")
            return df
        except Exception as e:
            logger.error(f"Error transforming sales data: {str(e)}")
            raise

# Sample usage
# Sample customer data
customer_data = {
    'customer_id': [1, 2, 3, 1],
    'email': ['alice@example.com', None, 'charlie@example.com', 'alice@example.com'],
    'phone': ['+1-800-555-1212', None, '555-1234', '+1-800-555-1212'],
    'registration_date': ['2023-01-01', '2023-02-15', '2023-03-20', '2023-01-01']
}
df_customer = pd.DataFrame(customer_data)

# Sample sales data
sales_data = {
    'sale_id': [101, 102, 103],
    'quantity': [2, 5, 1],
    'unit_price': [10.0, 20.0, 15.0],
    'discount_rate': [0.1, 0.2, 0.15],
    'sale_date': ['2023-04-01', '2023-05-12', '2023-06-23']
}
df_sales = pd.DataFrame(sales_data)

# Apply transformations
df_cleaned_customer = DataTransformer.clean_customer_data(df_customer)
df_transformed_sales = DataTransformer.transform_sales_data(df_sales)

# Display results
print("Cleaned Customer Data:")
print(df_cleaned_customer)

print("\nTransformed Sales Data:")
print(df_transformed_sales)


/var/folders/hf/wdvgwf0d7_91d_lts970z1jw0000gn/T/ipykernel_83801/3373813116.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['email'] = df['email'].fillna('')
/var/folders/hf/wdvgwf0d7_91d_lts970z1jw0000gn/T/ipykernel_83801/3373813116.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['phone'] = df['phone'].fillna('')
/var/folders/hf/wdvgwf0d7_91d_lts970z1jw0000gn/T/ipykernel_83801/3373813116.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

Cleaned Customer Data:
   customer_id                email        phone registration_date
0            1    alice@example.com  18005551212        2023-01-01
1            2                                          2023-02-15
2            3  charlie@example.com      5551234        2023-03-20

Transformed Sales Data:
   sale_id  quantity  unit_price  discount_rate  sale_date  total_amount  \
0      101         2        10.0           0.10 2023-04-01          20.0   
1      102         5        20.0           0.20 2023-05-12         100.0   
2      103         1        15.0           0.15 2023-06-23          15.0   

   discount_amount  final_amount  sale_year  sale_month  sale_quarter  
0             2.00         18.00       2023           4             2  
1            20.00         80.00       2023           5             2  
2             2.25         12.75       2023           6             2  


### ETL Pipeline
This class orchestrates the ETL pipeline, coordinating the extraction, transformation, and loading stages for both customer and sales data.

In [11]:
# ETLPipeline class
class ETLPipeline:
    def __init__(self):
        self.db_conn = DatabaseConnection()
        self.extractor = DataExtractor(self.db_conn)
        self.transformer = DataTransformer()
        self.loader = DataLoader(self.db_conn)
    
    def run_customer_pipeline(self):
        try:
            raw_data = self.extractor.extract_from_mongodb('customers')
            transformed_data = self.transformer.clean_customer_data(raw_data)
            self.loader.load_to_warehouse(transformed_data, 'dim_customer')
            logger.info("Customer pipeline completed successfully")
        except Exception as e:
            logger.error(f"Error in customer pipeline: {str(e)}")
            raise
    
    def run_sales_pipeline(self):
        try:
            raw_data = self.extractor.extract_from_api('sales')
            transformed_data = self.transformer.transform_sales_data(raw_data)
            self.loader.load_to_warehouse(transformed_data, 'fact_sales')
            logger.info("Sales pipeline completed successfully")
        except Exception as e:
            logger.error(f"Error in sales pipeline: {str(e)}")
            raise


### Running the Pipeline

In [8]:
etl_pipeline = ETLPipeline()
etl_pipeline.run_customer_pipeline()
etl_pipeline.run_sales_pipeline()

INFO:__main__:Mock DatabaseConnection initialized.
INFO:__main__:Mock MongoDB connection.
INFO:__main__:Mock load of 3 records to dim_customer
INFO:__main__:Customer pipeline completed successfully
INFO:__main__:Mock API extraction.
INFO:__main__:Mock load of 3 records to fact_sales
INFO:__main__:Sales pipeline completed successfully


In [22]:
def insert_supplier_to_sql(conn, supplier_name: str, contact_name: str, country: str):
    """Insert a supplier into the SQL database."""
    query = """
    INSERT INTO suppliers (supplier_name, contact_name, country) 
    VALUES (%s, %s, %s);
    """
    try:
        with conn.cursor() as cursor:
            cursor.execute(query, (supplier_name, contact_name, country))
            conn.commit()
            logger.info(f"Supplier '{supplier_name}' inserted successfully.")
    except Exception as e:
        logger.error(f"Error inserting data: {str(e)}")
        conn.rollback()
        raise

def get_suppliers_from_sql(conn):
    """Fetch supplier data from SQL database."""
    query = "SELECT * FROM suppliers WHERE country = 'USA';"
    try:
        # Use pandas to execute the query and return a DataFrame
        return pd.read_sql(query, conn)
    except Exception as e:
        logger.error(f"Error executing SQL query: {str(e)}")
        raise

In [ ]:
# SQL Connection setup
sql_conn = get_sql_connection(host="localhost", user="root", password="password", db="retail_db")

# Fetch data from SQL
df = get_suppliers_from_sql(sql_conn)
print(df.head())
